Lab 12 Task 
1. (Continue from Lab 11): 
• Develop Model’s Application Phase in Flask 
• Use ChatGPT for the frontend code 

In [1]:
from flask import Flask, request, jsonify

import pickle
import numpy as np

app = Flask(__name__)

# Load the trained model (from Lab 11)
with open("model/student_model.pkl", "rb") as f:
    model = pickle.load(f)

# Home route - serves frontend
@app.route("/")
def home():
    # Return frontend HTML directly
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Student Grade Predictor</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f2f2f2;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
            }
            .container {
                background: #fff;
                padding: 30px;
                border-radius: 10px;
                box-shadow: 0px 0px 10px #ccc;
                width: 400px;
            }
            input, select, button {
                width: 100%;
                padding: 10px;
                margin: 10px 0;
            }
            button {
                background-color: #4CAF50;
                color: white;
                border: none;
                cursor: pointer;
            }
            button:hover {
                background-color: #45a049;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Student Grade Prediction</h1>
            <form id="predictForm">
                <input type="number" step="0.1" name="study_hours" placeholder="Study Hours per Day" required>
                <input type="number" step="0.1" name="extracurricular" placeholder="Extracurricular Hours per Day" required>
                <input type="number" step="0.1" name="sleep_hours" placeholder="Sleep Hours per Day" required>
                <input type="number" step="0.1" name="social_hours" placeholder="Social Hours per Day" required>
                <input type="number" step="0.1" name="physical_activity" placeholder="Physical Activity Hours per Day" required>
                <select name="stress_level">
                    <option value="Low">Low</option>
                    <option value="Moderate">Moderate</option>
                    <option value="High">High</option>
                </select>
                <select name="gender">
                    <option value="Male">Male</option>
                    <option value="Female">Female</option>
                </select>
                <button type="submit">Predict</button>
            </form>
            <h2 id="result"></h2>
        </div>

        <script>
            const form = document.getElementById("predictForm");
            form.addEventListener("submit", async (e) => {
                e.preventDefault();
                const formData = new FormData(form);
                const data = Object.fromEntries(formData.entries());

                const response = await fetch("/predict", {
                    method: "POST",
                    headers: {"Content-Type": "application/json"},
                    body: JSON.stringify(data)
                });

                const result = await response.json();
                document.getElementById("result").innerText = result.predicted_grade 
                    ? `Predicted Grade: ${result.predicted_grade}`
                    : `Error: ${result.error}`;
            });
        </script>
    </body>
    </html>
    """

# Prediction route - backend logic
@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.json
        study_hours = float(data['study_hours'])
        extracurricular = float(data['extracurricular'])
        sleep_hours = float(data['sleep_hours'])
        social_hours = float(data['social_hours'])
        physical_activity = float(data['physical_activity'])
        stress_level = data['stress_level']
        gender = data['gender']

        # Encode categorical features
        stress_dict = {"Low": 0, "Moderate": 1, "High": 2}
        gender_dict = {"Male": 0, "Female": 1}
        stress_encoded = stress_dict.get(stress_level, 1)
        gender_encoded = gender_dict.get(gender, 0)

        X = np.array([[study_hours, extracurricular, sleep_hours, social_hours, physical_activity, stress_encoded, gender_encoded]])
        predicted_grade = model.predict(X)[0]

        return jsonify({"predicted_grade": round(predicted_grade, 2)})

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    app.run(debug=True)


FileNotFoundError: [Errno 2] No such file or directory: 'model/student_model.pkl'